In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gptnew/gpt.json
/kaggle/input/gpthuman/otvet.mail.ru/otvet.mail.ru/gpt.json
/kaggle/input/gpthuman/otvet.mail.ru/otvet.mail.ru/people.json


In [2]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.8 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [3]:
import json
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, XLMRobertaForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training, TaskType
from accelerate import Accelerator
from torch.utils.data import Dataset

# Инициализация Accelerate
accelerator = Accelerator()


In [4]:
import json
import pandas as pd
import re

# Функция для загрузки и предобработки данных с человеческими ответами
def load_human_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    cleaned_data = []
    for item in data:
        
        answer = f"{item['answer']['text']}"
        
        # Склеиваем и чистим текст
        combined_text = f"{answer}"
        combined_text = re.sub(r"[\"']", "", combined_text)  # Удаление кавычек
        combined_text = re.sub(r"\s+", " ", combined_text).strip()  # Удаление лишних пробелов
        cleaned_data.append({"id": item['id'], "text": combined_text, "target": 0})
    return pd.DataFrame(cleaned_data)

# Функция для загрузки и предобработки данных с машинными ответами
def load_machine_data(file_path, human_data_df):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)[0]  # Извлекаем первый элемент списка, который является словарем
    cleaned_data = []
    for id, text in data.items():
        if int(id) in human_data_df['id'].values:  # Проверка на соответствие ID

            
            # Склеиваем и чистим текст
            combined_text = f"{text}"
            combined_text = re.sub(r"[\"']", "", combined_text)  # Удаление кавычек
            combined_text = re.sub(r"\s+", " ", combined_text).strip()  # Удаление лишних пробелов
            cleaned_data.append({"id": int(id), "text": combined_text, "target": 1})
    return pd.DataFrame(cleaned_data)

# Путь к файлам
human_data_path = '/kaggle/input/gpthuman/otvet.mail.ru/otvet.mail.ru/people.json'
machine_data_path = '/kaggle/input/gptnew/gpt.json'

# Загрузка и обработка данных
human_df = load_human_data(human_data_path)
machine_df = load_machine_data(machine_data_path, human_df)

# Объединение данных
merged_df = pd.concat([human_df, machine_df], ignore_index=True)

# Проверка результата
print(merged_df.head())



        id                                               text  target
0  1079405  Есть французы!!! Perdu - потеряный , от perdre...       0
1  1079517  запомни: в последней фразе, которую ты скажешь...       0
2  1079574  Заговорит. Наверняка хоть что-то и как-то он п...       0
3  1079678  сосуд с мочой предвещает ухудшение здоровья и ...       0
4  1079681  METALLICA рулит!!! Что бы там всякие папазоглы...       0


dtype('int64')

In [5]:
import torch
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
from peft import LoraConfig, TaskType, prepare_model_for_kbit_training, get_peft_model
from torch.utils.data import Dataset
import pandas as pd

combined_data = merged_df[['text', 'target']].to_dict(orient='records')

class BinaryClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.texts = [item['text'] for item in data]
        self.labels = [item['target'] for item in data]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        tokens = {key: val.squeeze(0) for key, val in tokens.items()}  # Убираем первую размерность
        tokens['labels'] = torch.tensor(label, dtype=torch.long)
        return tokens

tokenizer = AutoTokenizer.from_pretrained("sagteam/xlm-roberta-large-sag")
model = XLMRobertaForSequenceClassification.from_pretrained('sagteam/xlm-roberta-large-sag', num_labels=2)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    layers_to_transform=[i for i in range(24) if i >= 15]
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

dataset = BinaryClassificationDataset(combined_data, tokenizer, max_length=512)

train_size = int(0.8 * len(dataset))
train_dataset, dev_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=8,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    optim="adamw_8bit",
    fp16=True,
    learning_rate=2e-4,
)

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    accuracy = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds)
    return {"accuracy": accuracy, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

# Подготовка модели и данных для использования с Accelerator
trainer.model, trainer.train_dataloader, trainer.eval_dataloader = accelerator.prepare(
    trainer.model, trainer.get_train_dataloader(), trainer.get_eval_dataloader()
)

# Обучение модели
trainer.train()

# Финальная оценка на валидационном наборе
final_eval_results = trainer.evaluate(eval_dataset=dev_dataset)
print(f"Final evaluation results on dev dataset: {final_eval_results}")



tokenizer_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sagteam/xlm-roberta-large-sag and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_30/2319582127.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.508600,0.284496,0.935948,0.938053
100,0.002700,0.241885,0.962092,0.961992
150,0.199900,0.194270,0.971242,0.970667
200,0.087000,0.471913,0.945098,0.946429
250,0.011300,0.212691,0.966013,0.966057
300,0.000300,0.135407,0.976471,0.975676
350,0.051800,0.134356,0.980392,0.980027
400,0.000700,0.117958,0.984314,0.984085
450,0.193200,0.102335,0.984314,0.984127
500,0.019600,0.102497,0.986928,0.986667


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [6]:
final_eval_results = trainer.evaluate(eval_dataset=dev_dataset)
print(f"Final evaluation results on dev dataset: {final_eval_results}")



Final evaluation results on dev dataset: {'eval_loss': 0.08950305730104446, 'eval_accuracy': 0.9869281045751634, 'eval_f1': 0.9867021276595745}


In [ ]:
lora_save_path = "/kaggle/working/"

# Сохранение только LoRA весов
model.save_pretrained(lora_save_path)
tokenizer.save_pretrained(lora_save_path)

In [7]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('output/checkpoint-400')


/kaggle/working


ValueError: Cannot display a directory using FileLink. Use FileLinks to display 'output/checkpoint-400'.

In [11]:
import torch
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification
from peft import PeftModel, PeftConfig

# Функция для инициализации модели и загрузки LoRA весов
def initialize_model_with_lora(base_model_name, lora_weights_path):
    # Загружаем базовую модель и токенизатор
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    model = XLMRobertaForSequenceClassification.from_pretrained(base_model_name, num_labels=2)
    
    # Загружаем LoRA-адаптеры
    model = PeftModel.from_pretrained(model, lora_weights_path)
    model.eval()  # Переводим модель в режим оценки (inference)
    return model, tokenizer

# Функция для обработки текста
def process_text(model, tokenizer, text, max_length=512):
    # Токенизация входного текста
    tokens = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    # Прогон через модель
    with torch.no_grad():
        outputs = model(**tokens)
    # Получение предсказаний
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).item()
    return predictions


In [ ]:
base_model_name = "sagteam/xlm-roberta-large-sag"
lora_weights_path = "/kaggle/working/output/checkpoint-500"


In [12]:

model, tokenizer = initialize_model_with_lora(base_model_name, lora_weights_path)
print("Model and tokenizer loaded successfully.")
prediction = process_text(model, tokenizer, user_input)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sagteam/xlm-roberta-large-sag and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully.


In [16]:
user_input = "Для того чтобы использовать код, который вы привели, необходимо создать объект combined_data из вашего DataFrame df, где будут использованы столбцы text и target. Далее мы можем преобразовать этот DataFrame в нужный формат — список словарей, где каждый словарь содержит text и target. Также потребуется импортировать необходимые модули, если они еще не импортированы."
prediction = process_text(model, tokenizer, user_input)

# Интерпретация результата
if prediction == 1:
    print("Классификация: Написан человеком")
else:
    print("Классификация: Сгенерирован")


Классификация: Сгенерирован
